Partie 1 : Tâche de Classification

In [ ]:

from bs4 import BeautifulSoup
import requests
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Scraping des données
urls = [
    "https://ar.wikipedia.org/wiki/الذكاء_الاصطناعي",
    "https://ar.wikipedia.org/wiki/تعلم_الآلة"
]

data = []
for url in urls:
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    paragraphs = soup.find_all('p')
    for p in paragraphs:
        text = p.get_text().strip()
        if len(text) > 50:  # Filtrer les petits paragraphes
            data.append(text)

# Texte de référence pour calculer la similarité
reference_text = "موضوع عن الذكاء الاصطناعي"

# TF-IDF vectorisation
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform([reference_text] + data)
similarities = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:]).flatten()

# Création du DataFrame avec les scores
df = pd.DataFrame({'Text': data, 'Score': similarities * 10})
df.to_csv('arabic_dataset1.csv', index=False, encoding='utf-8-sig')
print("Dataset créé avec scores : arabic_dataset1.csv")


Dataset créé avec scores : arabic_dataset1.csv


In [ ]:

from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string
import pyarabic.araby as araby
import nltk
from tashaphyne.stemming import ArabicLightStemmer

nltk.download('punkt')  # Tokenization
nltk.download('stopwords')  # Stopwords

# Charger le dataset
df = pd.read_csv('arabic_dataset1.csv', encoding='utf-8-sig')

# Pipeline de prétraitement
stemmer = ArabicLightStemmer()

def preprocess_text(text):
    text = araby.strip_tashkeel(text)
    text = araby.normalize_hamza(text)
    text = ''.join([char for char in text if char not in string.punctuation])
    tokens = word_tokenize(text)
    stop_words = set(stopwords.words('arabic'))
    tokens = [word for word in tokens if word not in stop_words]
    tokens = [stemmer.light_stem(word) for word in tokens]
    return ' '.join(tokens)

df['Cleaned_Text'] = df['Text'].apply(preprocess_text)
df.to_csv('preprocessed_dataset1.csv', index=False, encoding='utf-8-sig')
print("Prétraitement terminé, fichier : preprocessed_dataset1.csv")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Prétraitement terminé, fichier : preprocessed_dataset1.csv


In [ ]:
df = pd.read_csv('preprocessed_dataset1.csv', encoding='utf-8-sig')
print(df.head())

                                                Text     Score  \
0  الْذَكَاءُ الْاِصْطِنَاعِيُّ[2] أو الْذَكَاءُ ...  0.267774   
1  الذكاء الاصطناعي هو فرع من علم الحاسوب. تُعرِّ...  0.278761   
2  هذا التعريف، من حيث الأهداف والأفعال والتصور و...  0.188722   
3  تأسس هذا المجال على افتراض أن مَلَكة الذكاء يم...  0.261350   
4  إن بحوث الذكاء الاصطناعي من الأبحاث عالية التخ...  0.678202   

                                        Cleaned_Text  
0  ذكاء الاصطناعي2 ءو ذكاء صناعي3 ءو ذكاء صنعي4 ء...  
1  ذكاء اصطناع رع حاسوب عرف كثير مءلف كون « دراس ...  
2  تعريف، ءهداف ءفعال تصور بيء رجع ءلى Russell No...  
3  ءسس مجال فتراض ءن ملك ذكاء مك صف دق درج مك ءءل...  
4  ءن حوث ذكاء اصطناع ءبحاث عال تخصص تقنية، درج ء...  


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Bidirectional, GRU, LSTM, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np

# Charger le dataset prétraité
df = pd.read_csv('preprocessed_dataset.csv', encoding='utf-8-sig')

# Tokenization et padding
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(df['Cleaned_Text'])
sequences = tokenizer.texts_to_sequences(df['Cleaned_Text'])
max_len = 100
X = pad_sequences(sequences, maxlen=max_len, padding='post')
y = df['Score'] / 10  # Normalisation des scores

# Validation croisée pour les modèles
kf = KFold(n_splits=5, shuffle=True, random_state=42)

def build_model(architecture):
    model = Sequential([
        Embedding(input_dim=5000, output_dim=128, input_length=max_len),
        architecture,
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])
    return model

results = {"Model": [], "MAE": [], "MSE": []}

architectures = {
    "RNN": SimpleRNN(64),
    "Bidirectional RNN": Bidirectional(SimpleRNN(64)),
    "GRU": GRU(64),
    "LSTM": LSTM(64)
}

for name, architecture in architectures.items():
    maes, mses = [], []
    for train_index, val_index in kf.split(X):
        X_train, X_val = X[train_index], X[val_index]
        y_train, y_val = y[train_index], y[val_index]
        model = build_model(architecture)
        model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=5, batch_size=32, verbose=0)
        predictions = model.predict(X_val).flatten()
        maes.append(mean_absolute_error(y_val, predictions))
        mses.append(mean_squared_error(y_val, predictions))
    results["Model"].append(name)
    results["MAE"].append(np.mean(maes))
    results["MSE"].append(np.mean(mses))

# Résultats
results_df = pd.DataFrame(results)
print(results_df)


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 202ms/step


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 180ms/step


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


...

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 200ms/step


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 189ms/step


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 194ms/step
               Model       MAE       MSE
0                RNN  0.032953  0.001485
1  Bidirectional RNN  0.037008  0.001848
2                GRU  0.051932  0.005300
3               LSTM  0.028761  0.002299


In [ ]:
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

# Exemple d'évaluation avec BLEU Score
gold_standard = "الذكاء الاصطناعي هو دراسة الأنظمة الذكية"
predictions = ["الذكاء الاصطناعي يدرس الأنظمة الذكية"]

# Utilisation d'un lissage pour éviter les scores de 0
smoothing = SmoothingFunction().method1
bleu = sentence_bleu([gold_standard.split()], predictions[0].split(), smoothing_function=smoothing)
print(f"BLEU Score: {bleu:.4f}")


BLEU Score: 0.1316


In [ ]:
gold_standard = "التعلم الآلي هو فرع من الذكاء الاصطناعي"
predictions = ["التعلم الآلي هو أحد فروع الذكاء الاصطناعي"]

# Calcul du BLEU score
bleu = sentence_bleu([gold_standard.split()], predictions[0].split(), smoothing_function=smoothing)
print(f"BLEU Score: {bleu:.4f}")


BLEU Score: 0.2056


Partie 2 : Modèle Transformer (Génération de texte)

In [ ]:
pip install pyarabic


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.4/126.4 kB 10.1 MB/s eta 0:00:00


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from torch.utils.data import Dataset, DataLoader
from torch.optim import AdamW
from tqdm import tqdm
import requests
from bs4 import BeautifulSoup
import pyarabic.araby as araby
import string
import torch

# **1. Charger le modèle AraGPT2 et le tokenizer**
model_name = "aubmindlab/aragpt2-medium"  # Modèle optimisé pour l'arabe
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Ajouter un token de padding
tokenizer.pad_token = tokenizer.eos_token

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/843 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.50G [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.94M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.50M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/4.52M [00:00<?, ?B/s]

In [ ]:
# **2. Récupérer et prétraiter les données depuis Wikipédia**
def scrape_and_preprocess(url):
    # Récupérer les données de la page Wikipédia
    response = requests.get(url)
    response.raise_for_status()  # Vérifie que la requête a réussi
    soup = BeautifulSoup(response.text, 'html.parser')
    paragraphs = soup.find_all('p')

    # Extraire les paragraphes et appliquer un prétraitement
    data = []
    for para in paragraphs:
        text = para.get_text().strip()
        if len(text) > 50 and "المراجع" not in text:  # Filtrer les petits paragraphes et les sections inutiles
            text = araby.strip_tashkeel(text)  # Supprimer les voyelles courtes
            text = araby.normalize_hamza(text)  # Normaliser les hamzas
            text = ''.join([char for char in text if char not in string.punctuation])  # Supprimer la ponctuation
            data.append(text)
    return data

# URL des pages Wikipédia à scraper
urls = [
    "https://ar.wikipedia.org/wiki/الذكاء_الاصطناعي",
    "https://ar.wikipedia.org/wiki/تعلم_الآلة"
]

# Récupérer et concaténer toutes les données
dataset = []
for url in urls:
    dataset.extend(scrape_and_preprocess(url))

print(f"Nombre de paragraphes récupérés : {len(dataset)}")

# **Sauvegarder les paragraphes dans un fichier texte**
with open("paragraphs.txt", "w", encoding="utf-8") as f:
    for paragraph in dataset:
        f.write(paragraph + "\n\n")

print("Les paragraphes récupérés ont été sauvegardés dans le fichier 'paragraphs.txt'.")


Nombre de paragraphes récupérés : 76
Les paragraphes récupérés ont été sauvegardés dans le fichier 'paragraphs.txt'.


In [ ]:
# **3. Créer un DataLoader pour l'entraînement**
class TextDataset(Dataset):
    def __init__(self, texts, tokenizer, max_length=512):
        self.texts = texts
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        encodings = self.tokenizer(text, truncation=True, padding='max_length', max_length=self.max_length, return_tensors='pt')
        return encodings.input_ids.squeeze(), encodings.attention_mask.squeeze()

# Créer le dataset et le DataLoader
train_dataset = TextDataset(dataset, tokenizer)
train_dataloader = DataLoader(train_dataset, batch_size=2, shuffle=True)


In [ ]:
# **4. Entraîner le modèle**
# Configuration des hyperparamètres
epochs = 3
learning_rate = 5e-5

# Optimiseur
optimizer = AdamW(model.parameters(), lr=learning_rate)

# Entraînement
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.train()

for epoch in range(epochs):
    loop = tqdm(train_dataloader, leave=True)
    for batch in loop:
        input_ids, attention_mask = batch

        # Déplacer les données sur le GPU si disponible
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)

        # Calculer la perte
        outputs = model(input_ids, attention_mask=attention_mask, labels=input_ids)
        loss = outputs.loss

        # Rétropropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        # Mise à jour de l'affichage
        loop.set_description(f"Epoch {epoch + 1}/{epochs}")
        loop.set_postfix(loss=loss.item())

# Sauvegarder le modèle fine-tuné
model.save_pretrained('fine_tuned_aragpt2')
tokenizer.save_pretrained('fine_tuned_aragpt2')

print("Modèle entraîné et sauvegardé dans 'fine_tuned_aragpt2'.")

Epoch 3/3: 100%|██████████| 38/38 [00:34<00:00,  1.11it/s, loss=1.07]


Modèle entraîné et sauvegardé dans 'fine_tuned_aragpt2'.


In [ ]:
# **5. Génération de texte avec le modèle fine-tuné**
# Charger le modèle fine-tuné
model = AutoModelForCausalLM.from_pretrained('fine_tuned_aragpt2')
tokenizer = AutoTokenizer.from_pretrained('fine_tuned_aragpt2')
tokenizer.pad_token = tokenizer.eos_token

# Fonction de génération de texte
def generate_text(prompt, max_length=200, temperature=0.7, top_p=0.9):
    inputs = tokenizer(prompt, return_tensors='pt', padding=True, truncation=True, max_length=max_length)
    input_ids = inputs['input_ids'].to(model.device)
    attention_mask = inputs['attention_mask'].to(model.device)

    outputs = model.generate(
        input_ids,
        max_length=max_length,
        num_return_sequences=1,
        no_repeat_ngram_size=2,
        temperature=temperature,
        do_sample=True,
        top_p=top_p,
        pad_token_id=tokenizer.pad_token_id,
        attention_mask=attention_mask
    )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Exemple de génération
prompt =" تعريف الذكاء الاصطناعي "
generated_text = generate_text(prompt)
print(f"Texte généré : {generated_text}")

Texte généré :  تعريف الذكاء الاصطناعي مءسسات مثل الذكاء الصناعي، وشبكة واسعة من الشبكات العصبية؟ ءصبحت هذه الشبكات جزءا من شبكة معقدة من المشاكل التي تتطلب من العامل لحل هذه المشاكل؛ ولذلكؔ ءكثر من ذلكػ ءذا كان العامل قادرا على حل مشاكل محددة؅ ءو على سبيل المثال؏�ognition�ohom‏— naturalism�ata‟ Handward�AInection”�overy‛ مثل مشاكل فرعية في الرءيسية� vision–�ame computer vision�ame�
والشبكة العصبية الاصطناعية�ames recogbot recocnisation�ural recomotion��ertificial engines�riting intaction�ines�AI�ower in recordialism�umanism recoping�eed income innocural language�ord replantionism in social recruvig


In [ ]:
# **5. Génération de texte avec le modèle fine-tuné**
# Charger le modèle fine-tuné
model = AutoModelForCausalLM.from_pretrained('fine_tuned_aragpt2')
tokenizer = AutoTokenizer.from_pretrained('fine_tuned_aragpt2')
tokenizer.pad_token = tokenizer.eos_token

# Fonction de génération de texte
def generate_text(prompt, max_length=200, temperature=0.7, top_p=0.9):
    inputs = tokenizer(prompt, return_tensors='pt', padding=True, truncation=True, max_length=max_length)
    input_ids = inputs['input_ids'].to(model.device)
    attention_mask = inputs['attention_mask'].to(model.device)

    outputs = model.generate(
        input_ids,
        max_length=max_length,
        num_return_sequences=1,
        no_repeat_ngram_size=2,
        temperature=temperature,
        do_sample=True,
        top_p=top_p,
        pad_token_id=tokenizer.pad_token_id,
        attention_mask=attention_mask
    )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Exemple de génération
prompt ="الذكاء الاصطناعي هو  "
generated_text = generate_text(prompt)
print(f"Texte généré : {generated_text}")

Texte généré : الذكاء الاصطناعي هو  مءسس الذكاء الاصطناعي، ويجب على جميع البشر استخدام مفاهيم محددة مثل التفكير المنطقي وءذا كان ذلك ءو الذكاء هو ءيضا موجود من الناحية النظرية؟؛ ولذلك من الممكن ءن يعمل على تحسين ءبحاث الذكاء البشري؄ول مثل الذكاء الصناعي في المستقبلؔ ءصبحت ءكثر قدرة على معالجة المشاكل التي يواجهها البشر في الوقت الحاليػ؅ ءدى ذلك؏�ata‏— learning�
والاستدلال على ءلى قواعد معينة�riting pattern‟ ءيدمج الذكاء في المناهج الاجتماعية�ame intelligence�umanism؅ايرنجليزية search engines‛ 75
تستخدم في العديد من المجالات مثل الءنجليزي�uralism�ohotion” ءول مرة في عام 1974�ord�ight inbot revigion�eed recognition‌ وستانفورد Johns Wales‘�


In [ ]:
# **5. Génération de texte avec le modèle fine-tuné**
# Charger le modèle fine-tuné
model = AutoModelForCausalLM.from_pretrained('fine_tuned_aragpt2')
tokenizer = AutoTokenizer.from_pretrained('fine_tuned_aragpt2')
tokenizer.pad_token = tokenizer.eos_token

# Fonction de génération de texte
def generate_text(prompt, max_length=200, temperature=0.7, top_p=0.9):
    inputs = tokenizer(prompt, return_tensors='pt', padding=True, truncation=True, max_length=max_length)
    input_ids = inputs['input_ids'].to(model.device)
    attention_mask = inputs['attention_mask'].to(model.device)

    outputs = model.generate(
        input_ids,
        max_length=max_length,
        num_return_sequences=1,
        no_repeat_ngram_size=2,
        temperature=temperature,
        do_sample=True,
        top_p=top_p,
        pad_token_id=tokenizer.pad_token_id,
        attention_mask=attention_mask
    )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Exemple de génération
prompt ="شهدت أبحاث الذكاء الاصطناعي صحوة جديدة من خلال "
generated_text = generate_text(prompt)
print(f"Texte généré : {generated_text}")

Texte généré : شهدت أبحاث الذكاء الاصطناعي صحوة جديدة من خلال مءسسات التعليم في مجال الذكاء الصناعي، ومن خلال التركيز على المناهج التعليمية بشكل كبير على معالجة اللغة الطبيعية في مختلف المجالات؛ حيث قام الباحثون في جامعة برينستون وجامعة برينفيلد وجامعة كاليفورنيا وجامعة كارنيغي ميلون ومعهد ماساتشوستس للتكنولوجيا وجامعة ماساتشوستس وجامعة دارتموث وجامعة جورج هوبكنز ومعهد ماس للتكنولوجيا ومعهد كاليفورنيا للتكنولوجيا بتطوير المناهج الرياضية لتصور ءنماط محددة من المناهج في مجالات مختلفة مثل التعليم والتعلم الروبوتيات الرياضية؟ ءدوات رياضية محددة؅ ءكثر من ذلكػ في عام 1974ؔ ءواخرينيات؏�ines‏ianism�eeding— وهي ءبحاث علمية واسعة�ame vision‛ وبحلول عام 1982�ata recognition�uralism‟ recomnation�redialism record�AI recocomotion‌ وبحلول الثمانينيات؊طاني�93
والقواعد الرياضية التي تحدد السلوك الاجتماعي وفقا يءو «ءنها ءصبحت جزءا من
